In [1]:
import pandas as pd
import sklearn
import keras
import pickle

In [2]:
df = pd.read_csv('/content/df_model', index_col=0)
df.head()

,модель кат,тип двигателя кат,коробка передач кат,мощность мотора,количество владельцев,год выпуска,пробег,без дтп,цена
0,11,1,0,97,4,2010,182000,1,530000
1,1,1,1,122,3,2007,189000,1,550000
2,1,1,0,129,3,2013,111000,1,920000
3,1,1,1,115,2,2008,205000,1,470000
4,1,0,1,115,4,2008,197000,1,590000


In [3]:
train = df.drop(['цена'], axis=1)
target = df['цена']

In [4]:
from sklearn.model_selection import train_test_split
Xtrn, Xtest, Ytrn, Ytest = train_test_split(train, target, test_size=0.35)

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LogisticRegression
model1 = RandomForestRegressor(n_estimators=100, max_features ='sqrt')
model2 = KNeighborsRegressor(n_neighbors=8)
model3 = SVR(kernel='linear')
model4 = LogisticRegression()

In [6]:
from sklearn.metrics import r2_score

In [7]:
model1.fit(Xtrn, Ytrn) 
tmp1 = r2_score(Ytest, model1.predict(Xtest))
model2.fit(Xtrn, Ytrn) 
tmp2 = r2_score(Ytest, model2.predict(Xtest))
model3.fit(Xtrn, Ytrn) 
tmp3 = r2_score(Ytest, model3.predict(Xtest))
model4.fit(Xtrn, Ytrn) 
tmp4 = r2_score(Ytest, model4.predict(Xtest))

print(tmp1)
print(tmp2)
print(tmp3)
print(tmp4)

0.9166606195102087
0.5688583297752483
0.6201972477803676
0.24353556552266353


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [8]:
from keras.models import Sequential
from keras.layers import Dense

model5 = Sequential()
model5.add(Dense(32, input_dim=8, activation='relu'))
model5.add(Dense(16, activation='relu'))
model5.add(Dense(4, kernel_initializer='normal', activation='relu'))
model5.add(Dense(1, kernel_initializer='normal'))

In [9]:
model5.compile(loss='mean_squared_error', optimizer='Nadam')
model5.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 4)                 68        
                                                                 
 dense_3 (Dense)             (None, 1)                 5         
                                                                 
Total params: 889
Trainable params: 889
Non-trainable params: 0
_________________________________________________________________


In [10]:
from sklearn.preprocessing import StandardScaler
st = StandardScaler()
Xtrnew = st.fit_transform(Xtrn)
Ytrnew = Ytrn/10000000

In [11]:
model5.fit(Xtrnew, Ytrnew, epochs=170)

Epoch 1/170
34/34 [==============================] - 2s 3ms/step - loss: 0.0361
Epoch 2/170
34/34 [==============================] - 0s 3ms/step - loss: 0.0216
Epoch 3/170
34/34 [==============================] - 0s 3ms/step - loss: 0.0071
Epoch 4/170
34/34 [==============================] - 0s 3ms/step - loss: 0.0031
Epoch 5/170
34/34 [==============================] - 0s 3ms/step - loss: 0.0024
Epoch 6/170
34/34 [==============================] - 0s 3ms/step - loss: 0.0021
Epoch 7/170
34/34 [==============================] - 0s 3ms/step - loss: 0.0019
Epoch 8/170
34/34 [==============================] - 0s 4ms/step - loss: 0.0018
Epoch 9/170
34/34 [==============================] - 0s 3ms/step - loss: 0.0017
Epoch 10/170
34/34 [==============================] - 0s 4ms/step - loss: 0.0016
Epoch 11/170
34/34 [==============================] - 0s 3ms/step - loss: 0.0016
Epoch 12/170
34/34 [==============================] - 0s 3ms/step - loss: 0.0015
Epoch 13/170
34/34 [=================

In [12]:
Xtest_new = st.fit_transform(Xtest)
Ytest_new = Ytest/10000000

In [13]:
pred = model5.predict(Xtest_new).flatten()
tmp5 = r2_score(Ytest_new, pred)
print(tmp5)

15/15 [==============================] - 0s 2ms/step
0.8998066268686463


In [14]:
print(r2_score(Ytest, (pred * 10000000 + model1.predict(Xtest)) / 2))

0.8980261953296756


In [ ]:
model5.save("my_model_keras")

In [ ]:
pickle.dump(model1, open("my_model_RFR", 'wb'))